In [2]:
import os
import pandas as pd
import pendulum
import yfinance as yf

In [ ]:
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest 
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest
from alpaca.trading.enums import AssetStatus
from alpaca.trading.enums import AssetClass
from alpaca.data.enums import Adjustment
from alpaca.data.timeframe import TimeFrameUnit

In [ ]:
# Environment Variables
api_key = os.getenv("ALPACA_API_KEY")
secret_key = os.getenv("ALPACA_SECRET_KEY")

# Clients
data_client = StockHistoricalDataClient(api_key,secret_key)
trading_client = TradingClient(api_key,secret_key)

In [ ]:
asset_request = GetAssetsRequest(
    asset_class = AssetClass.US_EQUITY,
    status=AssetStatus.ACTIVE,
)

all_assets = trading_client.get_all_assets(asset_request)

all_assets = [vars(asset) for asset in all_assets]

df = pd.DataFrame(data=all_assets)

df = df[df['tradable'] & df['shortable'] & df['fractionable']]

data_path = "assets.csv"

df.to_csv(data_path,index=False)

In [ ]:
# Parameters
end = pendulum.datetime(2023,12,31)
start = pendulum.datetime(2020,12,31)

symbols = pd.read_csv('assets.csv')
symbols = symbols['symbol'].to_list()

bars_request = StockBarsRequest(
    symbol_or_symbols=symbols,
    timeframe=TimeFrame(1,TimeFrameUnit.Day),
    start=start,
    end=end,
    adjustment=Adjustment.ALL,
)

bars = data_client.get_stock_bars(bars_request)

df = bars.df

df = df.reset_index()


data_path = "data.csv"

df.to_csv(data_path,index=False)

df

In [3]:
bmk = yf.download('SPY', start='2021-01-01', end='2024-07-31')

bmk.to_csv('bmk.csv')

[*********************100%%**********************]  1 of 1 completed


In [1]:
from db import DB

db = DB()

df = db.get_historical_data()

df.to_csv("data.csv")

Connection to the PostgreSQL database was successful.
